In [400]:
import os
import pandas as pd


In [401]:
from google.colab import drive
drive.mount("/content/drive")

os.chdir('/content/drive/MyDrive/SI 630 project')
# os.listdir('.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [402]:


def get_reports(name, addfilename=False):
    df_list = []
    for root, dirs, files in os.walk('.'):
        if os.path.basename(root).startswith(name):
            for file in files:
                if file.endswith('.csv'):
                    df = pd.read_csv(os.path.join(root, file))

                    for split in ['train','dev','test']:
                    # for split in ['train','dev','test']:
                        df1 = df.loc[df.split==split]

                        if 'accuracy' in df1.columns:
                            micro_avg_f1 = df1.loc[df1['index'] == 'f1-score','accuracy'].item()
                        elif 'micro avg' in df1.columns:
                            micro_avg_f1 = df1.loc[df1['index'] == 'f1-score','micro avg'].item()
                        macro_avg_f1 = df1.loc[df1['index'] == 'f1-score','macro avg'].item()
                        weighted_avg_f1 = df1.loc[df1['index'] == 'f1-score','weighted avg'].item()

                        model_name = os.path.basename(root)
                        if addfilename:
                            model_name = file.replace('_classification_report','').replace('.csv','') + '_' + model_name
                        model_df = pd.DataFrame({'model': [model_name], 'split': [split], 'micro_avg_f1': [micro_avg_f1],
                                                'macro_avg_f1': [macro_avg_f1], 'weighted_avg_f1': [weighted_avg_f1]})

                        df_list.append(model_df)

    return pd.concat(df_list, ignore_index=True)
    

In [403]:
all_models_df = get_reports('flair_ner_model')
baseline_models_df = get_reports('baseline_ner_model',True)
baseline_meddra_models_df = get_reports('baseline_meddra_model',True)

In [404]:
split_to_use = 'test'

v2_models = all_models_df.loc[all_models_df.model.str.contains('v2') & (all_models_df.split == split_to_use)]
v3_models = all_models_df.loc[all_models_df.model.str.contains('v3') & (all_models_df.split == split_to_use)]
v2_baseline = baseline_models_df.loc[baseline_models_df.model.str.contains('v2') & (baseline_models_df.split == split_to_use)]
v3_baseline = baseline_models_df.loc[baseline_models_df.model.str.contains('v3') & (baseline_models_df.split == split_to_use)]

v2_baseline_meddra = baseline_meddra_models_df.loc[baseline_meddra_models_df.model.str.contains('v2') & (baseline_meddra_models_df.split == split_to_use)]
v3_baseline_meddra = baseline_meddra_models_df.loc[baseline_meddra_models_df.model.str.contains('v3') & (baseline_meddra_models_df.split == split_to_use)]

v2 = pd.concat([v2_models, v2_baseline],ignore_index=True).sort_values(['split','micro_avg_f1'],ascending = False)
v3 = pd.concat([v3_models, v3_baseline],ignore_index=True).sort_values(['split','micro_avg_f1'],ascending = False)

In [405]:
def reduce_table(df, tags, ver):
    df.model = df.model.str.replace(ver,'').str.replace(tags,'').str.replace('flair_ner_model__','').str.replace('flair__model_','flair_original').str.replace('__model_','')\
        .str.replace('no_rnn','flair_no_rnn').str.replace('finetune_glove','flair_finetune_glove').str.replace('flair_original_','')
    df.drop(['split','weighted_avg_f1','macro_avg_f1'],axis=1,inplace=True)
    df.reset_index(drop=True,inplace=True)

reduce_table(v2,'ner','v2')
reduce_table(v3,'ner','v3')
reduce_table(v2_baseline,'ner','v2')
reduce_table(v3_baseline,'ner','v3')
reduce_table(v2_baseline_meddra,'meddra','v2')
reduce_table(v3_baseline_meddra,'meddra','v3')

<ipython-input-405-099aa8c6ebb2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.model = df.model.str.replace(ver,'').str.replace(tags,'').str.replace('flair_ner_model__','').str.replace('flair__model_','flair_original').str.replace('__model_','')\
<ipython-input-405-099aa8c6ebb2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['split','weighted_avg_f1','macro_avg_f1'],axis=1,inplace=True)
<ipython-input-405-099aa8c6ebb2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [406]:
v2

,model,micro_avg_f1
0,xlm-roberta-large,0.732026
1,spanbert-large-cased,0.709150
2,Bio_ClinicalBERT,0.694915
3,flair_no_rnn,0.681485
4,flair_original,0.415788
5,flair_finetune_glove,0.083531
6,random_baseline,0.047144
7,mostfreq_baseline,0.044356


In [407]:
v3

,model,micro_avg_f1
0,xlm-roberta-large,0.769548
1,spanbert-large-cased,0.758708
2,Bio_ClinicalBERT,0.746886
3,flair_no_rnn,0.694210
4,flair_original,0.470888
5,flair_finetune_glove,0.338586
6,random_baseline,0.088712
7,mostfreq_baseline,0.081700


In [408]:
v2_baseline_meddra

,model,micro_avg_f1
0,random_baseline,0.000335
1,mostfreq_baseline,0.010462


In [409]:
v3_baseline_meddra

,model,micro_avg_f1
0,mostfreq_baseline,0.010629
1,random_baseline,0.000502


In [410]:
print(v2.style.to_latex().replace('_','\_'))

\begin{tabular}{llr}
 & model & micro\_avg\_f1 \\
0 & xlm-roberta-large & 0.732026 \\
1 & spanbert-large-cased & 0.709150 \\
2 & Bio\_ClinicalBERT & 0.694915 \\
3 & flair\_no\_rnn & 0.681485 \\
4 & flair\_original & 0.415788 \\
5 & flair\_finetune\_glove & 0.083531 \\
6 & random\_baseline & 0.047144 \\
7 & mostfreq\_baseline & 0.044356 \\
\end{tabular}



In [411]:
print(v3.style.to_latex().replace('_','\_'))

\begin{tabular}{llr}
 & model & micro\_avg\_f1 \\
0 & xlm-roberta-large & 0.769548 \\
1 & spanbert-large-cased & 0.758708 \\
2 & Bio\_ClinicalBERT & 0.746886 \\
3 & flair\_no\_rnn & 0.694210 \\
4 & flair\_original & 0.470888 \\
5 & flair\_finetune\_glove & 0.338586 \\
6 & random\_baseline & 0.088712 \\
7 & mostfreq\_baseline & 0.081700 \\
\end{tabular}



In [412]:
v2_baselines = pd.concat([v2_baseline,v2_baseline_meddra],ignore_index=True).sort_values(['micro_avg_f1'],ascending = False).reset_index(drop=True)
v2_baselines.model = ['ner_random','ner_mostfreq','meddra_random','meddra_mostfreq']
print(v2_baselines.style.to_latex().replace('_','\_'))

\begin{tabular}{llr}
 & model & micro\_avg\_f1 \\
0 & ner\_random & 0.047144 \\
1 & ner\_mostfreq & 0.044356 \\
2 & meddra\_random & 0.010462 \\
3 & meddra\_mostfreq & 0.000335 \\
\end{tabular}



In [413]:
v3_baselines = pd.concat([v3_baseline,v3_baseline_meddra],ignore_index=True).sort_values(['micro_avg_f1'],ascending = False).reset_index(drop=True)
v3_baselines.model = ['ner_random','ner_mostfreq','meddra_random','meddra_mostfreq']
print(v3_baselines.style.to_latex().replace('_','\_'))

\begin{tabular}{llr}
 & model & micro\_avg\_f1 \\
0 & ner\_random & 0.088712 \\
1 & ner\_mostfreq & 0.081700 \\
2 & meddra\_random & 0.010629 \\
3 & meddra\_mostfreq & 0.000502 \\
\end{tabular}

